In [62]:
pip install --upgrade pip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pickle
clf_lr = pickle.load(open('model.pkl', 'rb'))

In [2]:
from transformers import pipeline
fill_mask_pipeline = pipeline('fill-mask', model='distilroberta-base')
# fill_mask_pipeline("Hello I'm a <mask> model."), fill_mask_pipeline.model.lm_head

2023-05-06 15:05:51.709494: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
 fill_mask_pipeline.model

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [4]:
type(fill_mask_pipeline)

transformers.pipelines.fill_mask.FillMaskPipeline

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [6]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

CLASS_NAMES = [0, 1]
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

class EmbeddingsOfTextDataset(Dataset):
    def __init__(self, df, fill_mask_pipeline, class_names=CLASS_NAMES, device=device):
        self.df = df
        self.tokenizer = fill_mask_pipeline.tokenizer
        self.preprocessor_model = fill_mask_pipeline.model
        self.preprocessor_model.lm_head = nn.Identity()
        self.preprocessor_model = self.preprocessor_model.to(device)
        self.class_names = CLASS_NAMES
        super().__init__()
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        tweet_info = self.df.iloc[index]
        X = self.tokenizer(
            tweet_info['text'],
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )["input_ids"]
        
        
        X = X.to(device)
        with torch.no_grad():
            X = self.preprocessor_model.forward(X)[-1][0].sum(axis=0)

        t = X.to(torch.device("cpu"))
        del X
        print(f"{t.size() = }")
        return t, tweet_info["target"]

In [7]:
def GetPrediction(text):
    data = pd.DataFrame({'text': [text], 'target': 1})
    X = EmbeddingsOfTextDataset(data, fill_mask_pipeline).__getitem__(0)[0]
    p = float(clf_lr.predict_proba(X.reshape(1, -1))[:, 1]) 
    verdict = 'suicide' if float(clf_lr.predict_proba(X.reshape(1, -1))[:, 1]) >= 0.393939393939394 else 'not suicide'
    
    return (p, verdict)

In [11]:
GetPrediction("kekloool i wnt to die mazafaka")

t.size() = torch.Size([768])


(0.14886209513226203, 'not suicide')